In [1]:
import pandas as pd
import numpy as np
from knsmote import KMeansSMOTE
# from imblearn.over_sampling import KMeansSMOTE
from sklearn.cluster import MiniBatchKMeans
from collections import Counter
from data import get_data

In [2]:
# Define features and target variable
X, y, X_feature_names = get_data()

# Print original class distribution
print("Original class distribution:", Counter(y))

ValueError: blocks must be 2-D

In [ ]:
# Apply KMeansSMOTE to oversample minority classes
# Adjust KMeansSMOTE parameters
smote = KMeansSMOTE(
    kmeans_estimator=MiniBatchKMeans(n_clusters=20, n_init=1, random_state=0),  # Increase clusters
    cluster_balance_threshold=0.001,  # Lower threshold to allow smaller clusters
    sampling_strategy="auto",  # Try different values like {1: 500, 2: 500} for fine-tuning
    random_state=42
)

# Apply SMOTE
X_res, y_res = smote.fit_resample(X, y)

# Print new class distribution
print("Resampled class distribution:", Counter(y_res))

Resampled class distribution: Counter({3: 22510, 2: 22507, 0: 22506, 1: 22504, 4: 22503})


In [ ]:
X_res.shape

(112530, 287)

In [ ]:
len(X_feature_names.tolist())

287

In [ ]:
# Convert back to DataFrame and save the oversampled dataset
X_resampled_df = pd.DataFrame(X_res.toarray(), columns=X_feature_names.tolist())
y_resampled_df = pd.DataFrame(y_res, columns=["impact"])

# Combine resampled features with impact
resampled_df = pd.concat([y_resampled_df, X_resampled_df], axis=1)



In [ ]:
import h5py

In [ ]:
def save_data_withh5py(X_res, y_res):
    with h5py.File("output/features4ausw4linearsvc_trainsampled_v2.h5", "w") as f:
        # Save sparse matrix X as a compressed dataset
        f.create_dataset("X_data", data=X_res.data)
        f.create_dataset("X_indices", data=X_res.indices)
        f.create_dataset("X_indptr", data=X_res.indptr)
        f.create_dataset("X_shape", data=X_res.shape)

        # Save y as a dense dataset
        f.create_dataset("y", data=y_res)

save_data_withh5py(X_res, y_res)

AttributeError: 'numpy.ndarray' object has no attribute 'indices'